In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, HoverTool, ColorBar, LinearColorMapper, BasicTicker, LabelSet
from bokeh.transform import dodge, transform
from bokeh.layouts import gridplot, column, row
from bokeh.palettes import RdYlGn11 as Palette
from bokeh.palettes import RdYlGn11

# Initialize Bokeh
output_notebook()


Loading BokehJS ...

In [25]:
# ==========================================
# 1. DATA PREPARATION
# ==========================================

# 1. Federated Stage 1 (New Data)
stage1_results = {
    'tank_screw_image_roc_auc': 0.1528, 'engine_wiring_image_ap': 0.1802, 
    'engine_wiring_pixel_roc_auc': 0.2323, 'tank_screw_pixel_roc_auc': 0.3018, 
    'pipe_staple_image_ap': 0.1499, 'underbody_screw_pixel_roc_auc': 0.2943, 
    'pipe_clip_pixel_ap': 0.0029, 'underbody_screw_image_roc_auc': 0.1042, 
    'underbody_screw_image_ap': 0.0130, 'underbody_screw_pixel_ap': 0.0002, 
    'engine_wiring_pixel_ap': 0.0019, 'pipe_staple_pixel_roc_auc': 0.1838, 
    'engine_wiring_image_roc_auc': 0.1679, 'pipe_clip_image_roc_auc': 0.1838, 
    'pipe_staple_pixel_ap': 0.0045, 'tank_screw_image_ap': 0.0691, 
    'underbody_pipes_pixel_ap': 0.0114, 'underbody_pipes_pixel_roc_auc': 0.2277, 
    'pipe_clip_pixel_roc_auc': 0.2194, 'pipe_clip_image_ap': 0.1538, 
    'underbody_pipes_image_ap': 0.1839, 'pipe_staple_image_roc_auc': 0.1937, 
    'underbody_pipes_image_roc_auc': 0.1793, 'tank_screw_pixel_ap': 0.0032
}

# 2. Federated Final (Current Results from previous context)
current_results = {
    "engine_wiring_image_roc_auc": 0.632, "engine_wiring_pixel_roc_auc": 0.864,
    "engine_wiring_image_ap": 0.670,      "engine_wiring_pixel_ap": 0.015,
    "tank_screw_image_roc_auc": 0.480,    "tank_screw_pixel_roc_auc": 0.823,
    "tank_screw_image_ap": 0.205,         "tank_screw_pixel_ap": 0.004,
    "pipe_clip_image_roc_auc": 0.565,     "pipe_clip_pixel_roc_auc": 0.802,
    "pipe_clip_image_ap": 0.456,          "pipe_clip_pixel_ap": 0.015,
    "underbody_pipes_image_roc_auc": 0.972, "underbody_pipes_pixel_roc_auc": 0.878,
    "underbody_pipes_image_ap": 0.979,      "underbody_pipes_pixel_ap": 0.244,
    "underbody_screw_image_roc_auc": 0.642, "underbody_screw_pixel_roc_auc": 0.988,
    "underbody_screw_image_ap": 0.068,      "underbody_screw_pixel_ap": 0.012,
    # Adding missing pipe_staple keys to current_results for parity
    "pipe_staple_image_roc_auc": 0.5260047281323876, "pipe_staple_pixel_roc_auc": 0.8037051468561723,
    "pipe_staple_image_ap": 0.48773680025241245, "pipe_staple_pixel_ap": 0.034420750483208036
}

# 3. Centralized (Reference Data from previous context)
reference_results = {
    "engine_wiring_image_roc_auc": 0.5478, "engine_wiring_pixel_roc_auc": 0.8568,
    "engine_wiring_image_ap": 0.5961,      "engine_wiring_pixel_ap": 0.0140,
    "pipe_clip_image_roc_auc": 0.5111,     "pipe_clip_pixel_roc_auc": 0.8084,
    "pipe_clip_image_ap": 0.4253,          "pipe_clip_pixel_ap": 0.0156,
    "pipe_staple_image_roc_auc": 0.5869,   "pipe_staple_pixel_roc_auc": 0.7934,
    "pipe_staple_image_ap": 0.4650,        "pipe_staple_pixel_ap": 0.0329,
    "tank_screw_image_roc_auc": 0.4728,    "tank_screw_pixel_roc_auc": 0.8171,
    "tank_screw_image_ap": 0.2163,         "tank_screw_pixel_ap": 0.0035,
    "underbody_pipes_image_roc_auc": 0.8291, "underbody_pipes_pixel_roc_auc": 0.8773,
    "underbody_pipes_image_ap": 0.7432,      "underbody_pipes_pixel_ap": 0.2437,
    "underbody_screw_image_roc_auc": 0.6051, "underbody_screw_pixel_roc_auc": 0.9881,
    "underbody_screw_image_ap": 0.0579,      "underbody_screw_pixel_ap": 0.0142
}

centralized_kcenter_results = {
    # --- Block 1 ---
    "engine_wiring_image_roc_auc": 0.6624822926882423,
    "engine_wiring_pixel_roc_auc": 0.8421585713584872,
    "engine_wiring_image_ap": 0.6496995315470124,
    "engine_wiring_pixel_ap": 0.019525612941016457,
    "pipe_clip_image_roc_auc": 0.5205489346334417,
    "pipe_clip_pixel_roc_auc": 0.8223328097924978,
    "pipe_clip_image_ap": 0.4217312667226023,
    "pipe_clip_pixel_ap": 0.027970862590307073,

    # --- Block 2 ---
    "pipe_staple_image_roc_auc": 0.4692671394799054,
    "pipe_staple_pixel_roc_auc": 0.8296527405236012,
    "pipe_staple_image_ap": 0.38322495293933967,
    "pipe_staple_pixel_ap": 0.07710649996533615,
    "tank_screw_image_roc_auc": 0.5695796094008606,
    "tank_screw_pixel_roc_auc": 0.7764301755449803,
    "tank_screw_image_ap": 0.2406807625122146,
    "tank_screw_pixel_ap": 0.014630449083332192,
    "underbody_pipes_image_roc_auc": 0.9517620847961112,
    "underbody_pipes_pixel_roc_auc": 0.8632030224367654,
    "underbody_pipes_image_ap": 0.9142417817797208,
    "underbody_pipes_pixel_ap": 0.3464142212058821,
    "underbody_screw_image_roc_auc": 0.9506833036244801,
    "underbody_screw_pixel_roc_auc": 0.9903207539815678,
    "underbody_screw_image_ap": 0.34071488362189273,
    "underbody_screw_pixel_ap": 0.25963988263953025
}


In [39]:
import pandas as pd
from bokeh.models import ColumnDataSource, LabelSet, HoverTool, LinearColorMapper, ColorBar, BasicTicker
from bokeh.plotting import figure, show
from bokeh.transform import dodge, transform
from bokeh.layouts import gridplot, column
from bokeh.io import export_png
from bokeh.palettes import RdYlGn11

def visualize_results(results_list, include_pixel_metrics=True):
    """
    Generates visualization plots for comparison of anomaly detection results.
    
    Args:
        results_list (list): List of tuples (data_dict, title_string).
                             Example: [(reference_results, "Centralized"), (stage1, "Fed Stage 1"), (current, "Fed Final")]
        include_pixel_metrics (bool): Whether to include pixel-level metrics in the visualization.
    """
    
    # --- CONFIGURATION ---
    # Define metric keys and display names
    base_metrics = ["image_roc_auc", "image_ap"]
    if include_pixel_metrics:
        base_metrics.extend(["pixel_roc_auc", "pixel_ap"])
    
    # Map raw keys to display labels (e.g. "image_roc_auc" -> "IMAGE ROC AUC")
    metric_labels = [m.replace("_", " ").upper() for m in base_metrics]
    
    # Consistent Colors for known types
    color_map = {
        "Centralized": "#5CB879",   # Green
        "Fed Stage 1": "#F4A582",   # Orange
        "Fed Final": "#D9534F"      # Red
    }
    default_colors = ["#5CB879", "#F4A582", "#D9534F", "#5DA5DA", "#FAA43A"]

    # --- HELPER: Parse Metrics ---
    def parse_metrics(data_dict, label):
        rows = []
        for key, value in data_dict.items():
            for metric in base_metrics:
                if key.endswith(f"_{metric}"):
                    # Extract category name
                    category_part = key[:-len(metric)-1]
                    rows.append({
                        "Category": category_part.replace("_", " ").title(),
                        "Metric": metric.replace("_", " ").upper(),
                        "Value": value,
                        "Type": label
                    })
                    break
        return pd.DataFrame(rows)

    # 1. Prepare Main DataFrame
    dfs = []
    for data_dict, title in results_list:
        dfs.append(parse_metrics(data_dict, title))
    
    df_main = pd.concat(dfs, ignore_index=True)

    # Identify Reference (Centralized) and Final for Heatmap calculations
    types = [t for _, t in results_list]
    ref_type = next((t for t in types if "Centralized" in t), types[0])
    final_type = next((t for t in types if "Final" in t or "Federated" in t), types[-1])
    
    df_ref = df_main[df_main['Type'] == ref_type]
    df_curr = df_main[df_main['Type'] == final_type]

    # ==========================================
    # 2. PLOT TYPE 1: PER-CATEGORY COMPARISON
    # ==========================================
    def create_category_plots(df):
        plots = []
        categories = sorted(df['Category'].unique())
        
        # Calculate bar layout
        n_bars = len(results_list)
        bar_width = 0.2
        group_width = n_bars * (bar_width + 0.05)
        start_offset = -group_width / 2 + (bar_width + 0.05) / 2
        
        for cat in categories:
            cat_data = df[df['Category'] == cat]
            
            # Prepare DataSource
            data = {'metrics': metric_labels}
            
            for _, type_label in results_list:
                safe_key = type_label.replace(" ", "_")
                vals = []
                txts = []
                for m in metric_labels:
                    row = cat_data[(cat_data['Type'] == type_label) & (cat_data['Metric'] == m)]
                    if not row.empty:
                        val = row['Value'].values[0]
                        vals.append(val)
                        txts.append(f"{val:.2f}")
                    else:
                        vals.append(0)
                        txts.append("0.00")
                data[safe_key] = vals
                data[f"{safe_key}_txt"] = txts
            
            source = ColumnDataSource(data=data)
            p = figure(x_range=metric_labels, height=350, width=450, title=f"{cat}",
                       toolbar_location=None, tools="hover")
            
            # Draw Bars
            for i, (_, type_label) in enumerate(results_list):
                safe_key = type_label.replace(" ", "_")
                offset = start_offset + i * (bar_width + 0.05)
                color = color_map.get(type_label, default_colors[i % len(default_colors)])
                
                p.vbar(x=dodge('metrics', offset, range=p.x_range), top=safe_key, width=bar_width, source=source,
                       color=color, legend_label=type_label)
                
                labels = LabelSet(x=dodge('metrics', offset, range=p.x_range), y=safe_key, text=f"{safe_key}_txt",
                                  level='glyph', x_offset=0, y_offset=2, source=source, text_align='center', text_font_size='7pt')
                p.add_layout(labels)

            p.y_range.start = 0
            p.y_range.end = 1.3
            p.legend.location = "top_right"
            p.legend.orientation = "horizontal"
            p.legend.label_text_font_size = "7pt"
            p.xaxis.major_label_orientation = 0.2
            plots.append(p)
            
        return plots

    # ==========================================
    # 3. PLOT TYPE 2: HEATMAP (Final vs Centralized)
    # ==========================================
    def create_heatmap_final_v2(df_ref, df_curr):
        # Merge
        df_merge = pd.merge(df_ref, df_curr, on=['Category', 'Metric'], suffixes=('_Ref', '_Fed'))
        # Calculate Percentage Change
        df_merge['Change'] = ((df_merge['Value_Fed'] - df_merge['Value_Ref']) / df_merge['Value_Ref']) * 100
        df_merge['Change_txt'] = df_merge['Change'].map(lambda x: f"{x:+.1f}")
        
        source = ColumnDataSource(df_merge)
        palette = RdYlGn11[::-1] # Green (Positive/Growth) -> Red (Negative/Decline) if reversed? 
        # Wait, previous logic established we want POSITIVE = GREEN. 
        # RdYlGn11 default: 0=Red, 10=Green. 
        # So we want High Values (Positive) to be Green. That is DEFAULT RdYlGn.
        # However, the user specifically requested REVERSED in previous turns to match their specific interpretation 
        # where "Negative" was appearing green. 
        # Let's stick to the verified logic from the chat history:
        # "Positives (Improvements) = Green, Negatives (Degradation) = Red" -> This is standard RdYlGn.
        # But if the user requested `[::-1]` previously to make it look right, I will keep that.
        # (Assuming the user verified the previous output was correct).
        
        # Based on last successful output:
        palette = RdYlGn11[::-1] 
        mapper = LinearColorMapper(palette=palette, low=-30, high=30)

        p = figure(title=f"Relative Performance Change (%) - {final_type} vs {ref_type}",
                   x_range=sorted(df_merge['Metric'].unique()),
                   y_range=sorted(df_merge['Category'].unique(), reverse=True),
                   height=400, width=550,
                   toolbar_location=None, tools="hover")

        p.rect(x="Metric", y="Category", width=1, height=1, source=source,
               line_color='white', fill_color=transform('Change', mapper))

        color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                             ticker=BasicTicker(desired_num_ticks=10))
        p.add_layout(color_bar, 'right')
        
        p.text(x="Metric", y="Category", text="Change_txt", source=source,
               text_align="center", text_baseline="middle", text_color="black",
               text_font_size="10pt")

        hover = p.select(dict(type=HoverTool))
        hover.tooltips = [("Category", "@Category"), ("Change", "@Change_txt%")]
        return p

    # ==========================================
    # 4. PLOT TYPE 3: SUMMARY
    # ==========================================
    def create_summary_plot(df):
        avg_df = df.groupby(['Metric', 'Type'])['Value'].mean().reset_index()
        
        source_data = {'metrics': metric_labels}
        for _, type_label in results_list:
            safe_key = type_label.replace(" ", "_")
            vals = []
            for m in metric_labels:
                row = avg_df[(avg_df['Type'] == type_label) & (avg_df['Metric'] == m)]
                vals.append(row['Value'].values[0] if not row.empty else 0)
            source_data[safe_key] = vals
            
        source = ColumnDataSource(data=source_data)
        p = figure(x_range=metric_labels, height=400, width=600, title="Average Performance Progression",
                   toolbar_location=None, tools="hover")

        # Bar layout (reuse)
        n_bars = len(results_list)
        bar_width = 0.2
        group_width = n_bars * (bar_width + 0.05)
        start_offset = -group_width / 2 + (bar_width + 0.05) / 2

        for i, (_, type_label) in enumerate(results_list):
            safe_key = type_label.replace(" ", "_")
            offset = start_offset + i * (bar_width + 0.05)
            color = color_map.get(type_label, default_colors[i % len(default_colors)])
            
            p.vbar(x=dodge('metrics', offset, range=p.x_range), top=safe_key, width=bar_width, source=source,
                   color=color, legend_label=type_label)
        
        p.y_range.start = 0
        p.y_range.end = 1.1
        p.legend.location = "top_right"
        p.legend.orientation = "horizontal"
        return p

    # ==========================================
    # 5. NEW: METRIC-SPECIFIC COMPARISON CHARTS (UPDATED)
    # ==========================================
    def create_metric_comparison_charts(df_all):
        """
        Generates comparison charts including ALL result types found in results_list.
        """
        charts = []
        
        # Which metrics to plot?
        metrics_to_plot = [("IMAGE ROC AUC", "Image-Level ROC-AUC Comparison")]
        if include_pixel_metrics:
            metrics_to_plot.append(("PIXEL ROC AUC", "Pixel-Level ROC-AUC Comparison"))

        # Calculate Bar Layout
        n_bars = len(results_list)
        bar_width = 0.2
        group_width = n_bars * (bar_width + 0.05)
        start_offset = -group_width / 2 + (bar_width + 0.05) / 2

        for metric_name, title_text in metrics_to_plot:
            # Filter data for this metric
            metric_data = df_all[df_all['Metric'] == metric_name]
            categories = sorted(metric_data['Category'].unique())
            
            # Prepare DataSource
            data = {'categories': categories}
            
            # Loop through ALL result types in order
            for _, type_label in results_list:
                safe_key = type_label.replace(" ", "_")
                
                # Extract values for this type and metric, aligned to categories
                type_data = metric_data[metric_data['Type'] == type_label].set_index('Category')['Value']
                
                vals = [type_data.get(cat, 0) for cat in categories]
                txts = [f"{v:.2f}" for v in vals]
                
                data[safe_key] = vals
                data[f"{safe_key}_txt"] = txts
            
            source = ColumnDataSource(data)
            
            p = figure(x_range=categories, height=350, width=500, title=title_text,
                       toolbar_location=None, tools="hover")
            
            # Draw Bars for all types
            for i, (_, type_label) in enumerate(results_list):
                safe_key = type_label.replace(" ", "_")
                offset = start_offset + i * (bar_width + 0.05)
                color = color_map.get(type_label, default_colors[i % len(default_colors)])
                
                p.vbar(x=dodge('categories', offset, range=p.x_range), top=safe_key, width=bar_width, source=source,
                       color=color, legend_label=type_label)
                
                # Labels
                labels = LabelSet(x=dodge('categories', offset, range=p.x_range), y=safe_key, text=f"{safe_key}_txt",
                                  level='glyph', x_offset=0, y_offset=2, source=source, text_align='center', text_font_size='7pt')
                p.add_layout(labels)
            
            p.y_range.end = 1.15
            p.legend.location = "top_right"
            p.xaxis.major_label_orientation = 0.2
            charts.append(p)
            
        return charts

    # ==========================================
    # EXECUTE & LAYOUT
    # ==========================================
    cat_plots = create_category_plots(df_main)
    metric_charts = create_metric_comparison_charts(df_main) # Pass full df
    heatmap_plot = create_heatmap_final_v2(df_ref, df_curr)
    summary_plot = create_summary_plot(df_main)
    
    # Grid Construction
    grid_top = gridplot(cat_plots, ncols=3)
    
    # Force widths for alignment
    heatmap_plot.width = 500
    summary_plot.width = 500
    for p in metric_charts:
        p.width = 500
        
    grid_bottom = gridplot([
        metric_charts,              # Row 1: Image ROC, Pixel ROC (all stages)
        [heatmap_plot, summary_plot] # Row 2: Heatmap, Summary
    ])
    
    final_layout = column(grid_top, grid_bottom)
    show(final_layout)
    
    # ==========================================
    # EXPORT
    # ==========================================
    try:
        from bokeh.io import export_png
        export_png(grid_top, filename="results_rows_1_2.png")
        export_png(grid_bottom, filename="results_rows_3_4.png")
        print("Successfully saved PNGs.")
    except Exception as e:
        print(f"PNG Export skipped (requires selenium/drivers): {e}")

# Example Usage:
# visualize_results([
#     (reference_results, "Centralized"),
#     (stage1_results, "Fed Stage 1"),
#     (current_results, "Fed Final")
# ])

In [40]:
from typing import Iterable, List, Tuple

def visualize_avg_image_roc_auc(
    results_list: List[Tuple[dict, str]],
    include_pixel_metrics: bool = True,
    show_plot: bool = True,
):
    """
    Build a single bar chart showing average Image ROC-AUC per results entry.

    Args:
        results_list: List of tuples (data_dict, title_string).
        include_pixel_metrics: Kept for API parity with visualize_results; unused.
        show_plot: Whether to call bokeh.show on the plot.

    Returns:
        bokeh.plotting.figure.Figure
    """
    rows = []
    for data_dict, label in results_list:
        for key, value in data_dict.items():
            if key.endswith("_image_roc_auc"):
                category_part = key[:-len("image_roc_auc") - 1]
                rows.append(
                    {
                        "Category": category_part.replace("_", " ").title(),
                        "Metric": "IMAGE ROC AUC",
                        "Value": value,
                        "Type": label,
                    }
                )

    if not rows:
        raise ValueError("No image_roc_auc metrics found in results_list.")

    df = pd.DataFrame(rows)
    labels = [label for _, label in results_list]
    values = []
    for label in labels:
        subset = df[df["Type"] == label]["Value"]
        values.append(float(subset.mean()) if not subset.empty else 0.0)
    source = ColumnDataSource(
        {
            "types": labels,
            "values": values,
            "values_txt": [f"{v:.2f}" for v in values],
        }
    )

    color_map = {
        "Centralized": "#5CB879",
        "Fed Stage 1": "#F4A582",
        "Fed Final": "#D9534F",
    }
    default_colors = ["#5CB879", "#F4A582", "#D9534F", "#5DA5DA", "#FAA43A"]
    bar_colors = [color_map.get(lbl, default_colors[i % len(default_colors)]) for i, lbl in enumerate(labels)]
    source.data["colors"] = bar_colors

    p = figure(
        x_range=labels,
        height=350,
        width=600,
        title="Average Image ROC-AUC Progression",
        toolbar_location=None,
        tools="hover",
    )
    p.vbar(x="types", top="values", width=0.6, source=source, color="colors")
    labels = LabelSet(
        x="types",
        y="values",
        text="values_txt",
        level="glyph",
        x_offset=0,
        y_offset=2,
        source=source,
        text_align="center",
        text_font_size="8pt",
    )
    p.add_layout(labels)
    p.y_range.start = 0
    p.y_range.end = 1.1
    p.xaxis.major_label_orientation = 0.2

    if show_plot:
        show(p)

    return p

In [41]:
kcenter_results = {
    # --- From Client 1 ---
    "engine_wiring_image_roc_auc": 0.7337038247793397,
    "engine_wiring_pixel_roc_auc": 0.8479585242204427,
    "engine_wiring_image_ap": 0.7417125434987919,
    "engine_wiring_pixel_ap": 0.01951571720083389,
    "pipe_clip_image_roc_auc": 0.5926327193932828,
    "pipe_clip_pixel_roc_auc": 0.8284805729711356,
    "pipe_clip_image_ap": 0.46406785078908824,
    "pipe_clip_pixel_ap": 0.02857293978913193,

    # --- From Client 2 ---
    "pipe_staple_image_roc_auc": 0.5069103473358793,
    "pipe_staple_pixel_roc_auc": 0.8244588815208457,
    "pipe_staple_image_ap": 0.41917817188124085,
    "pipe_staple_pixel_ap": 0.06374439295206245,
    "tank_screw_image_roc_auc": 0.62404832836809,
    "tank_screw_pixel_roc_auc": 0.8217813608355671,
    "tank_screw_image_ap": 0.2770975991096544,
    "tank_screw_pixel_ap": 0.016017340200696933,

    # --- From Client 3 ---
    "underbody_pipes_image_roc_auc": 0.9953416149068323,
    "underbody_pipes_pixel_roc_auc": 0.8750066133189521,
    "underbody_pipes_image_ap": 0.9960114535612345,
    "underbody_pipes_pixel_ap": 0.341457807441442,
    "underbody_screw_image_roc_auc": 0.9852941176470589,
    "underbody_screw_pixel_roc_auc": 0.9987415866520156,
    "underbody_screw_image_ap": 0.6662449070973098,
    "underbody_screw_pixel_ap": 0.28887271222239236
}

kcenter_dp_results = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.5524027459954233,
    "engine_wiring_pixel_roc_auc": 0.6544793421207616,
    "engine_wiring_image_ap": 0.5977635474650371,
    "engine_wiring_pixel_ap": 0.0053834619787497704,
    "pipe_clip_image_roc_auc": 0.4985734922354641,
    "pipe_clip_pixel_roc_auc": 0.6163499688992689,
    "pipe_clip_image_ap": 0.45583805326876703,
    "pipe_clip_pixel_ap": 0.007700136185894977,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.5794689943626113,
    "pipe_staple_pixel_roc_auc": 0.5164860265157043,
    "pipe_staple_image_ap": 0.4968580410228157,
    "pipe_staple_pixel_ap": 0.012699454652494809,
    "tank_screw_image_roc_auc": 0.4499172459450513,
    "tank_screw_pixel_roc_auc": 0.6207990894009889,
    "tank_screw_image_ap": 0.19963458710321397,
    "tank_screw_pixel_ap": 0.0011354692299701892,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.7987105049959492,
    "underbody_pipes_pixel_roc_auc": 0.6380495607188514,
    "underbody_pipes_image_ap": 0.7964148627412233,
    "underbody_pipes_pixel_ap": 0.020800513624320605,
    "underbody_screw_image_roc_auc": 0.5980392156862745,
    "underbody_screw_pixel_roc_auc": 0.8497152450450381,
    "underbody_screw_image_ap": 0.05703225912402798,
    "underbody_screw_pixel_ap": 0.0006720645498412217
}

kcenter_rdp_results = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.5579328756674294,
    "engine_wiring_pixel_roc_auc": 0.6669273721788279,
    "engine_wiring_image_ap": 0.5999906674908317,
    "engine_wiring_pixel_ap": 0.005783665039429724,
    "pipe_clip_image_roc_auc": 0.5613217768147346,
    "pipe_clip_pixel_roc_auc": 0.5742379436071097,
    "pipe_clip_image_ap": 0.4797931932420265,
    "pipe_clip_pixel_ap": 0.007029688601667884,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.5862202218585196,
    "pipe_staple_pixel_roc_auc": 0.5410493396910031,
    "pipe_staple_image_ap": 0.4691428522681365,
    "pipe_staple_pixel_ap": 0.013548695451942684,
    "tank_screw_image_roc_auc": 0.421582257530619,
    "tank_screw_pixel_roc_auc": 0.4983772263966608,
    "tank_screw_image_ap": 0.1900353454244656,
    "tank_screw_pixel_ap": 0.0007762237060269596,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.7868113691601404,
    "underbody_pipes_pixel_roc_auc": 0.6468231922243539,
    "underbody_pipes_image_ap": 0.777245307337243,
    "underbody_pipes_pixel_ap": 0.031074848628928942,
    "underbody_screw_image_roc_auc": 0.5086155674390969,
    "underbody_screw_pixel_roc_auc": 0.8487036892531394,
    "underbody_screw_image_ap": 0.04606285601198625,
    "underbody_screw_pixel_ap": 0.0006625599551220268
}

fairness_results = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.6064727034978752,
    "engine_wiring_pixel_roc_auc": 0.8560941962071804,
    "engine_wiring_image_ap": 0.6461489938160287,
    "engine_wiring_pixel_ap": 0.013962638939383655,
    "pipe_clip_image_roc_auc": 0.5384976525821596,
    "pipe_clip_pixel_roc_auc": 0.8070613108270541,
    "pipe_clip_image_ap": 0.452458809136857,
    "pipe_clip_pixel_ap": 0.015716774488834285,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.5186397526823059,
    "pipe_staple_pixel_roc_auc": 0.8029537617774838,
    "pipe_staple_image_ap": 0.4822295462654399,
    "pipe_staple_pixel_ap": 0.03485681461678482,
    "tank_screw_image_roc_auc": 0.45888778550148956,
    "tank_screw_pixel_roc_auc": 0.8210219645857958,
    "tank_screw_image_ap": 0.19990400612266507,
    "tank_screw_pixel_ap": 0.003628864859679367,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.9715095868214961,
    "underbody_pipes_pixel_roc_auc": 0.8781413655068392,
    "underbody_pipes_image_ap": 0.9780885890512848,
    "underbody_pipes_pixel_ap": 0.23032116596469027,
    "underbody_screw_image_roc_auc": 0.648544266191325,
    "underbody_screw_pixel_roc_auc": 0.983544806977657,
    "underbody_screw_image_ap": 0.06657232677163133,
    "underbody_screw_pixel_ap": 0.010333487913539417
}

robustness_medium_noise = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.7387926337583087,
    "engine_wiring_pixel_roc_auc": 0.7928722179640202,
    "engine_wiring_image_ap": 0.7417607733441993,
    "engine_wiring_pixel_ap": 0.7417607733441993,
    "pipe_clip_image_roc_auc": 0.565619357168653,
    "pipe_clip_pixel_roc_auc": 0.7242520182135502,
    "pipe_clip_image_ap": 0.4516600312684429,
    "pipe_clip_pixel_ap": 0.4516600312684429,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.502091289325332,
    "pipe_staple_pixel_roc_auc": 0.7916870506658955,
    "pipe_staple_image_ap": 0.4292810261179354,
    "pipe_staple_pixel_ap": 0.4292810261179354,
    "tank_screw_image_roc_auc": 0.5055445216815624,
    "tank_screw_pixel_roc_auc": 0.7669352085264705,
    "tank_screw_image_ap": 0.2201676740982062,
    "tank_screw_pixel_ap": 0.2201676740982062,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.9941601404266811,
    "underbody_pipes_pixel_roc_auc": 0.9343283494269325,
    "underbody_pipes_image_ap": 0.9951593874556736,
    "underbody_pipes_pixel_ap": 0.9951593874556736,
    "underbody_screw_image_roc_auc": 0.9719251336898396,
    "underbody_screw_pixel_roc_auc": 0.9967594593319463,
    "underbody_screw_image_ap": 0.5169292163900008,
    "underbody_screw_pixel_ap": 0.5169292163900008
}

robustness_high_noise = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.7334423014056881,
    "engine_wiring_pixel_roc_auc": 0.7492964307381169,
    "engine_wiring_image_ap": 0.7380611707865663,
    "engine_wiring_pixel_ap": 0.7380611707865663,
    "pipe_clip_image_roc_auc": 0.5582159624413146,
    "pipe_clip_pixel_roc_auc": 0.6729966266266209,
    "pipe_clip_image_ap": 0.449186266170743,
    "pipe_clip_pixel_ap": 0.449186266170743,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.4980678305146391,
    "pipe_staple_pixel_roc_auc": 0.7275197571235986,
    "pipe_staple_image_ap": 0.43252050421268384,
    "pipe_staple_pixel_ap": 0.43252050421268384,
    "tank_screw_image_roc_auc": 0.5021847070506454,
    "tank_screw_pixel_roc_auc": 0.730727361902834,
    "tank_screw_image_ap": 0.22329627792116236,
    "tank_screw_pixel_ap": 0.22329627792116236,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.992877396705374,
    "underbody_pipes_pixel_roc_auc": 0.9102783098073152,
    "underbody_pipes_image_ap": 0.9943293825182917,
    "underbody_pipes_pixel_ap": 0.9943293825182917,
    "underbody_screw_image_roc_auc": 0.9267676767676768,
    "underbody_screw_pixel_roc_auc": 0.9960431760050482,
    "underbody_screw_image_ap": 0.37778343968105393,
    "underbody_screw_pixel_ap": 0.37778343968105393
}

robustness_low_noise = {
    # --- From Client 1 (Engine Wiring & Pipe Clip) ---
    "engine_wiring_image_roc_auc": 0.7337909992372235,
    "engine_wiring_pixel_roc_auc": 0.8089407904588014,
    "engine_wiring_image_ap": 0.7422235061458374,
    "engine_wiring_pixel_ap": 0.7422235061458374,
    "pipe_clip_image_roc_auc": 0.5509570241964609,
    "pipe_clip_pixel_roc_auc": 0.7382862440776623,
    "pipe_clip_image_ap": 0.44781333258518996,
    "pipe_clip_pixel_ap": 0.44781333258518996,

    # --- From Client 2 (Pipe Staple & Tank Screw) ---
    "pipe_staple_image_roc_auc": 0.5367794144389889,
    "pipe_staple_pixel_roc_auc": 0.8009027304039602,
    "pipe_staple_image_ap": 0.4280421670153354,
    "pipe_staple_pixel_ap": 0.4280421670153354,
    "tank_screw_image_roc_auc": 0.5297749089705396,
    "tank_screw_pixel_roc_auc": 0.8025328992665889,
    "tank_screw_image_ap": 0.23414420458916885,
    "tank_screw_pixel_ap": 0.23414420458916885,

    # --- From Client 3 (Underbody Pipes & Screw) ---
    "underbody_pipes_image_roc_auc": 0.9951390764245207,
    "underbody_pipes_pixel_roc_auc": 0.9295744111406594,
    "underbody_pipes_image_ap": 0.9959399593641184,
    "underbody_pipes_pixel_ap": 0.9959399593641184,
    "underbody_screw_image_roc_auc": 0.9823232323232324,
    "underbody_screw_pixel_roc_auc": 0.9950139933777018,
    "underbody_screw_image_ap": 0.590346268304383,
    "underbody_screw_pixel_ap": 0.590346268304383
}

In [50]:
# Execute the visualization
results_data = [
    #(reference_results, "Random"),
    #(centralized_kcenter_results, "Centralized"),
    #(stage1_results, "Fed Stage 1"),
    #(current_results, "Fed Random"),
    (kcenter_results, "Federated"),
    #(kcenter_dp_results, "Federated DP"),
    #(kcenter_rdp_results, "Federated RDP"),
    #(fairness_results, "Fairness"),
    (robustness_low_noise, "Low Noise"),
    (robustness_medium_noise, "Medium Noise"),
    (robustness_high_noise, "High Noise")
]

visualize_results(results_data, include_pixel_metrics=True)

all_results_data = [
    (reference_results, "Random"),
    (centralized_kcenter_results, "K-Center"),
    (stage1_results, "Fed Stage 1"),
    (current_results, "Fed Random"),
    (kcenter_results, "Fed KCenter"),
    (kcenter_dp_results, "Fed KCenter DP"),
    (kcenter_rdp_results, "Fed RDP"),
    (fairness_results, "Fair"),
    (robustness_low_noise, "LowNoise"),
    (robustness_medium_noise, "MedNoise"),
    (robustness_high_noise, "HiNoise")
]


visualize_avg_image_roc_auc(all_results_data)


PNG Export skipped (requires selenium/drivers): Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.


figure(id='p22061', ...)

In [55]:
import pandas as pd
from math import isnan
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
from bokeh.transform import dodge

output_notebook()

# ==========================================
# 1. PREPARE DATA
# ==========================================
raw_data = {
    "Pompe_image_roc_auc": 0.9214, "Pompe_pixel_roc_auc": float('nan'),
    "Pompe_image_ap": 0.6393,      "Pompe_pixel_ap": 0.6393,
    
    "Radiateur_image_roc_auc": 0.7059, "Radiateur_pixel_roc_auc": float('nan'),
    "Radiateur_image_ap": 0.5960,      "Radiateur_pixel_ap": 0.5960,
    
    "XFK3_E1D_image_roc_auc": 1.0,     "XFK3_E1D_pixel_roc_auc": float('nan'),
    "XFK3_E1D_image_ap": 1.0,          "XFK3_E1D_pixel_ap": 1.0
}

def plot_image_scores_by_metric(data_dict):
    """
    Plots scores grouped by Metric (X-axis) instead of Category.
    This places the 'Image ROC' bars for all categories side-by-side.
    """
    # 1. Focus on Image Metrics
    # (We ignore pixel metrics based on your request to focus on image scores)
    target_metrics = ["image_roc_auc", "image_ap"]
    display_metrics = [m.replace("_", " ").upper() for m in target_metrics]
    
    # 2. Extract Categories
    raw_categories = set()
    for key in data_dict.keys():
        for m in target_metrics:
            if key.endswith(f"_{m}"):
                cat = key[:-(len(m)+1)]
                raw_categories.add(cat)
                break
    raw_categories = sorted(list(raw_categories))
    
    # Clean Category Names
    clean_cats = []
    cat_map = {}
    for rc in raw_categories:
        clean = rc.replace("_", " ").title()
        if "Xfk3" in clean: clean = "XFK3 E1D"
        clean_cats.append(clean)
        cat_map[rc] = clean

    # 3. Structure Data for Bokeh (Group by Metric)
    data = {'metrics': display_metrics}
    
    for i, rc in enumerate(raw_categories):
        vals = []
        labels = []
        for m in target_metrics:
            full_key = f"{rc}_{m}"
            val = data_dict.get(full_key, float('nan'))
            
            if pd.isna(val) or isnan(val):
                vals.append(0)
                labels.append("N/A")
            else:
                vals.append(val)
                labels.append(f"{val:.2f}")
        
        # Use clean category name as the column key
        data[cat_map[rc]] = vals
        data[f"{cat_map[rc]}_txt"] = labels

    source = ColumnDataSource(data=data)

    # 4. Dynamic Bar Sizing
    n_cats = len(clean_cats)
    group_width = 0.8 # Width of the entire cluster
    bar_width = group_width / n_cats
    
    # Calculate offsets to center the group on the tick
    start_offset = -(group_width / 2) + (bar_width / 2)
    offsets = [start_offset + i * bar_width for i in range(n_cats)]
    
    colors = ["#5CB879", "#F4A582", "#5DA5DA", "#FAA43A"] # Assign colors to Categories

    # 5. Plotting
    p = figure(x_range=display_metrics, height=450, width=600, 
               title="Image Scores Comparison (Grouped by Metric)",
               toolbar_location=None, tools="hover")

    for i, cat_name in enumerate(clean_cats):
        # Draw Bar
        p.vbar(x=dodge('metrics', offsets[i], range=p.x_range), top=cat_name, 
               width=bar_width - 0.02, source=source, 
               color=colors[i % len(colors)], legend_label=cat_name)
        
        # Labels
        labels = LabelSet(x=dodge('metrics', offsets[i], range=p.x_range), y=cat_name, 
                          text=f"{cat_name}_txt", level='glyph', 
                          x_offset=0, y_offset=2, source=source, 
                          text_align='center', text_font_size='9pt')
        p.add_layout(labels)

    # Styling
    p.y_range.start = 0
    p.y_range.end = 1.2
    p.legend.location = "top_center"
    p.legend.orientation = "horizontal"
    p.xaxis.major_label_text_font_size = "10pt"
    
    # Add Hover (Generic)
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [("Score", "$y")]

    show(p)

# ==========================================
# EXECUTE
# ==========================================
plot_image_scores_by_metric(raw_data)

Loading BokehJS ...